In [151]:
from pymongo import MongoClient
import json
import pandas as pd
import requests
from pandas import DataFrame
import numpy as np
import googlemaps
import pprint
import time
import requests
import folium
from folium import plugins

import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("G_CLOUD_API_KEY")

In [152]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [153]:
#cleaned_companies
companies = pd.DataFrame(list(db.companies_coor.find({'$and':[{'founded_year':{ "$ne" : None }},
                                                              {'latitud':{ "$ne" : None }}]},{"products":0,
                                                                                        "twitter_username":0,
                                                      "level_0":0,"_id":0, "founded_day":0, "founded_month":0}
                                                    )))

#cleaned_neighbors (for the moment working with NY neighbors)
neighbors = pd.DataFrame(list(db.neighborhood.find({})))
neighbors.head(3)

,_id,geometry,name
0,55cb9c666c522cafdb053a1a,"{'coordinates': [[[-73.94193078816193, 40.7007...",Bedford
1,55cb9c666c522cafdb053a1b,"{'coordinates': [[[-73.94732672160579, 40.6291...",Midwood
2,55cb9c666c522cafdb053a1c,"{'coordinates': [[[-73.89138023380261, 40.8617...",Fordham South


In [154]:
companies.rename(index=str, columns={"currency_n": "money_raised"}, inplace=True)
companies = companies[["index","name","category_code","founded_year",
                       "money_raised","latitud","longitud","city","location"]]
companies.head(3)

,index,name,category_code,founded_year,money_raised,latitud,longitud,city,location
0,7,StumbleUpon,web,2002.0,1.85e+07,37.775196,-122.419204,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,..."
1,3,Geni,web,2006.0,1.65e+07,34.090368,-118.393064,West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,9,Slacker,music,2006.0,7.31e+07,33.022176,-117.081406,San Diego,"{'type': 'Point', 'coordinates': [-117.081406,..."


def findNear(geopoint, radio_max_meters=200):
    return db.companies_coor.find({
        "location": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })
geopoint = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]  #NY, 133 offices
}
city = "city"
radio_max_meters = 1000
num_offices = findNear(geopoint, 1000).count()
print(f"Hay {num_offices} oficinas cerca de {city} a {radio_max_meters} metros")

In [155]:
#Ranking weighted averages:
    # 1   category_code
    # 2   closest companies
    # 3   founded year < 10  

In [156]:
#Ranking weighted averages   #1   category_code
def pondera_v(cat): 
    suma = 0
    if cat == "games_video":
        suma = 4
    if cat == "web":
        suma = 3
    if cat == "music":
        suma == 2
    return suma
companies ['wa_category'] = companies["category_code"].apply(pondera_v)

In [159]:
#Ranking weighted averages   #2   closest companies

def findNear(geopoint, radio_max_meters=200):
    return db.companies_coor.find({
        "location": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })

def findNear_category(df, radio_max_meters=1000):
    #filter by city if desired
    #city = input(" Enter city: ")
    #df_filtered = df[df['city']==city]
    
    lstng = []
    for row in range(len(df)):
        geopoint = df.location.iloc[row]
        
        #sort by closest companies from loc  
        lst = len(list(findNear(geopoint,)))
        lstng.append(lst)
        df_overSF=pd.DataFrame(lstng)
    return lstng, df_overSF

lstng_a, df_overSF = (findNear_category(companies))
# Group by “source”, and count the total number of “source”

In [160]:
#df_overSF = df_overSF.reset_index(drop=True)
df_overSF.head()

,0
0,59
1,1
2,1
3,19
4,39


In [161]:
companies["founded_year"] = companies["founded_year"].astype(int)
companies.head(2)

,index,name,category_code,founded_year,money_raised,latitud,longitud,city,location,wa_category
0,7,StumbleUpon,web,2002,1.85e+07,37.775196,-122.419204,San Francisco,"{'type': 'Point', 'coordinates': [-122.419204,...",3
1,3,Geni,web,2006,1.65e+07,34.090368,-118.393064,West Hollywood,"{'type': 'Point', 'coordinates': [-118.393064,...",3


In [162]:
#Ranking weighted averages   #3   founded year

def find_founded_year(df, radio_max_meters=2000):
    lista=[]
    for row in range(len(df)): 
        old=db.companies_coor.find({'$and':[{
                                    "location": {
                                     "$near": {
                                       "$geometry": df[row],
                                       "$maxDistance": radio_max_meters,
                                     }
                                   }
                                },
        {"founded_year": {"$gt":2003}
        }]})
                                
        df_=pd.DataFrame(old)
        lista.append(len(df_))
        df_founded=pd.DataFrame(lista)
    return lista,df_founded

lista, df_f_year = find_founded_year(companies['location'], 2000)

In [163]:
df_f_year.head()

,0
0,139
1,7
2,2
3,245
4,203


In [164]:
companies.reset_index(drop=True, inplace=True)
df_overSF.reset_index(drop=True, inplace=True)
df_f_year.reset_index(drop=True, inplace=True)

In [165]:
compan = pd.concat([companies,df_overSF,df_f_year], axis=1, ignore_index=False)

In [180]:
compan['total'] = compan.iloc[:, -4:-1].sum(axis=1)
compan.sort_values(by=['average'], ascending=False)

In [196]:
mapa = folium.Map([20,-20], zoom_start=1.5)
location = compan[['latitud', 'longitud']].head(200).values
mapa.add_child(plugins.HeatMap(location, radius=15))
mapa

In [ ]:
                                                        #       MEDIR DISTANCIAS   ---- LATER ON
def distances_c(df,radio_max_meters=10000):  
    lista = []
    for i in range(len(df)):
        a=db.companies_coor.aggregate([
        { "$geoNear": {
            "near": {                    # near: The point for which to find the closest documents.
                "$geoemtry": df[i]
            }, 
            "maxDistance": radio_max_meters,    #The maximum distance from the center point that the documents can be
            "spherical": False,
            "distanceField": "distance" , # The output field that contains the calculated distance.
            "query": { "category_code": "web"},
            "num": 1,
        }}
        ])
        b=pd.DataFrame(a)
        lista.append(b)
    return lista
    
    
distances = list(distances_c(companies,))
print(distances)

In [ ]:
# https://developers.google.com/places/web-service/search

In [ ]:
# GOOGLEPLACES --- 
def get_lat_long_places(name):   # name as bus stops
    try:
        query = "{}".format(name)
        print(query)
        # location format :   -33.8670522,151.1957362  tengo que meter latitud, longitud
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type=restaurant&keyword=cruise&key={}".format(latitud,longitud,radius,key)
        response = requests.get(url)
        data = response.json()
        location = data.get('results')[0].get('geometry').get('location')
        return{
            "latitud" : location.get('lat'),
            "longitud" : location.get('lng')
        }
    except: 
        return {
            "latitud" : None,
            "longitud" : None
        }

In [ ]:
                                            # Future venture:
# REQUEST TRAFFIC API
    https://traffic.api.here.com/traffic/6.2/flow/xml/8/134/86
    ?app_id={YOUR_APP_ID}
    &app_code={YOUR_APP_CODE}
#https://developer.here.com/documentation/traffic/topics_v6.1/example-flow.html